In [1]:
import pandas as pd
import numpy as np
import re
import regex


In [2]:
df = pd.read_csv('../Data/Original/merged.csv', 
                 sep=',', 
                 encoding='utf-8', 
                 dtype=object)
# df.head()


In [3]:
print(df['right_person_name'].nunique())
print(df['reg_num'].nunique())
print(df['ipc'].nunique())

391536
6302339
64690


In [4]:
# 実験場
# list(df[df['right_person_name'].str.contains('\?ｫ|▲', regex=True)]\
#         [['right_person_name']]\
#         .drop_duplicates(subset=['right_person_name'], keep='first')\
#         .sort_values('right_person_name', ascending=True)['right_person_name'].unique())#.head(30)
# adate_df[adate_df['right_person_name'].str.contains('キヤノン')]['right_person_name'].unique()
# df[df['right_person_name'].str.startswith('愛知県')]['right_person_name'].unique()


In [5]:
# 旧字体の辞書作る用のデータ
old_new_letter_df = pd.read_html(
    'https://www.asahi-net.or.jp/~ax2s-kmtn/ref/old_chara.html',
    encoding='utf-8'
)

df_tmp_list = []
for i in [1, 2]:
    for c in range(2, 14+1, 6):
        df_tmp = old_new_letter_df[i].iloc[:, c:c+1+1]
        df_tmp.columns = ['new_jikei', 'old_jikei']
        df_tmp_list.append(df_tmp)

old_new_letter_df = pd.concat(df_tmp_list, 
                              axis='index')\
                      .dropna(how='all')\
                      .reset_index(drop=True)


In [6]:
company_master_df = pd.read_csv('../../0_Data/3_NISTEP/Data/Dealed/company_master.csv', encoding='utf-8', sep=',')
company_master_df.head()


,before_name,after_name
0,アーカイラス・エンジニアリング合資会社,アーカイラス・エンジニアリング合資会社
1,株式会社アーキス,株式会社アーキス
2,アーキテクツ・スタジオ・ジャパン株式会社,アーキテクツ・スタジオ・ジャパン株式会社
3,ＴＥＭＰＯ ＮＥＴＷＯＲＫ株式会社,アーキテクツ・スタジオ・ジャパン株式会社
4,レンタル保証システム株式会社,アーキテクツ・スタジオ・ジャパン株式会社


In [7]:
trans_specialstr_tuple = (
    ('?b', '高'), 
    ('?C', '吉'), 
    ('?D', '塚'), 
    ('?F', '崎'), 
    ('?H', '徳'), 
    ('?P', '濾'), 
    ('?芟ｴ', '桑原'), 
    ('??', '')
)

In [8]:
for trans_specialstr in trans_specialstr_tuple:
    df['right_person_name'] = df['right_person_name'].str.replace(trans_specialstr[0], trans_specialstr[1], regex=False)

In [9]:
list(df[df['right_person_name'].str.contains('??', regex=False)]['right_person_name'].unique())

[]

## 一文字の違いシリーズ

In [10]:
# 旧字体の辞書
old_new_letter_dict = dict(zip(old_new_letter_df['old_jikei'].values, 
                               old_new_letter_df['new_jikei'].values))

# 消す文字のリスト
trans_noise_list = ['\u3000', '\?ｫ', '\?ｬ', '▲', '▼', ' ']

# 一文字:一文字の置換
trans_one_to_one_dict = {
                'ャ': 'ヤ', 
                'ョ': 'ヨ', 
                'ュ': 'ユ', 
                'ァ': 'ア', 
                'ィ': 'イ', 
                'ゥ': 'ウ', 
                'ェ': 'エ', 
                'ォ': 'オ', 
                'ッ': 'ツ', 
                'ヶ': 'ケ', 
                '―': 'ー',  
                '－': 'ー',
                '－': 'ー'
                }


In [11]:
# 旧字体を新字体に変換，消す文字を消す
for col_name in ['right_person_addr', 'right_person_name']:
    df[col_name] = df[col_name].str.replace('|'.join(trans_noise_list), '', regex=True)\
                                                 .str.translate(str.maketrans(old_new_letter_dict))\
                                                 .str.translate(str.maketrans(trans_one_to_one_dict))


In [12]:
for col_name in ['before_name', 'after_name']:
    company_master_df[col_name] = company_master_df[col_name].str.replace('|'.join(trans_noise_list), '', regex=True)\
                                                             .str.translate(str.maketrans(old_new_letter_dict))\
                                                             .str.translate(str.maketrans(trans_one_to_one_dict))


---


## 省庁とか


In [13]:
# 省庁合併のタプルを作る用のデータ
minis_office_df = pd.read_csv('../../0_Data/4_Ministry/Data/Dealed/minis_office.csv', 
                              encoding='utf-8', sep=',', dtype=object)
minis_office_df.head()


,old_office,after_office
0,.*地方創生推進事務.*,内閣府
1,.*知的財産戦略推進事務.*,内閣府
2,.*科学技術・イノベーション推進事務.*,内閣府
3,.*健康・医療戦略推進事務.*,内閣府
4,.*宇宙開発戦略推進事務.*,内閣府


In [14]:
# 大臣とかの修正
trans_minister_tuple = tuple([('大臣', '省')])\
                       +tuple(zip(minis_office_df['old_office'].values, minis_office_df['after_office'].values))
# trans_minister_tuple


In [16]:
exception_list = []
i = 0
for trans_minister in trans_minister_tuple:
    if i in [0, 31, 37, 47, 51, 52, 56, 61, 63, 73, 74, 88, 89, 91, 92, 93, 94, 96, 97, 98, 100, 102, 104, 109, 110, 112, 115, 116, 117, 118, 119, 120, 121, 122, 123]:
        exception_list.append(df[df['right_person_name'].str.contains(trans_minister[0])]['right_person_name'].str.replace(trans_minister[0], '').unique())
        df['right_person_name'] = df['right_person_name'].str.replace(trans_minister[0], trans_minister[1], regex=True)
    i += 1
# df


---


## 官庁とか何かの長とか

In [17]:
# 何らかの長の表記ゆれ
trans_top_tuple = (
               ('総長|長官', ''), 
               ('所長', '所'), 
               ('局長', '局'), 
               ('院長', '院'), 
               ('大学{1,2}長', '大学'), 
               ('校長', '校'), 
               ('課長', '課'), 
               ('部長', '部'), 
               ('センター長', 'センター'), 
               ('機構長', '機構'), 
               ('署長', '署'), 
               ('場長', '場'), 
               ('市長', '市')
               )

# 企業名の表記ゆれ
trans_lp_tuple = (('コーポレイシヨン', 'コーポレーシヨン'), 
                  ('株式会社', '株式'), 
                  ('株式', '株式会社'))

trans_else_tuple = (
               ('パナソニツクＩＰマネジメント株式会社', 'パナソニツク株式会社'), 
               ('ッ', 'ツ')
                    )

In [18]:
for trans_top in trans_top_tuple+trans_lp_tuple:
    df['right_person_name'] = df['right_person_name'].str.replace(trans_top[0], trans_top[1], regex=True)
    for col_name in ['before_name', 'after_name']:
        company_master_df[col_name] = company_master_df[col_name].str.replace(trans_top[0], trans_top[1], regex=True)
df.head()


,reg_num,app_year_month_day,set_reg_year_month_day,right_person_addr,right_person_name,ipc
0,5684381,20110708,20150123,ドイツ連邦共和国,フラウンホツフアーーゲゼルシヤフトツアーフエーデルングデアアンゲバンテンフオルシユングエーフアー,G01S 5/14
1,5684381,20110708,20150123,ドイツ連邦共和国,フリードリツヒーアレクサンダーーユニバーシタツタエルランゲンーニユルンベルク,G01S 5/14
2,5684399,20120209,20150123,アメリカ合衆国,ザジレツトカンパニーリミテツドライアビリテイカンパニー,B26B 21/52
3,5684492,20100512,20150123,東京都渋谷区,ＤＲＣ合同会社,G10H 3/18
4,5684500,20100623,20150123,アメリカ合衆国,バイエル・マテリアルサイエンス・リミテツド・ライアビリテイ・カンパニー,C08G 65/26


In [19]:
# # 合併
df = pd.merge(df, company_master_df.rename(columns={'before_name':'right_person_name'}), 
              on='right_person_name', 
              how='left')\
             .rename(columns={'after_name':'after_right_person_name'})
df['after_right_person_name'] = df['after_right_person_name'].fillna(df['right_person_name'])
df = df.drop(columns=['right_person_name'])\
       .rename(columns={'after_right_person_name':'right_person_name'})
df.head()


,reg_num,app_year_month_day,set_reg_year_month_day,right_person_addr,ipc,right_person_name
0,5684381,20110708,20150123,ドイツ連邦共和国,G01S 5/14,フラウンホツフアーーゲゼルシヤフトツアーフエーデルングデアアンゲバンテンフオルシユングエーフアー
1,5684381,20110708,20150123,ドイツ連邦共和国,G01S 5/14,フリードリツヒーアレクサンダーーユニバーシタツタエルランゲンーニユルンベルク
2,5684399,20120209,20150123,アメリカ合衆国,B26B 21/52,ザジレツトカンパニーリミテツドライアビリテイカンパニー
3,5684492,20100512,20150123,東京都渋谷区,G10H 3/18,ＤＲＣ合同会社
4,5684500,20100623,20150123,アメリカ合衆国,C08G 65/26,バイエル・マテリアルサイエンス・リミテツド・ライアビリテイ・カンパニー


---


In [20]:
# df.to_csv('../Data/Tmp/fixed.csv', encoding='utf-8', sep=',', index=False)
df.to_csv('../Data/Tmp/fixed_merged.csv', encoding='utf-8', sep=',', index=False)
